In [1]:
import pandas as pd
from tqdm import tqdm
from fuzzywuzzy import fuzz
from unidecode import unidecode
import re
from concurrent.futures import ProcessPoolExecutor

/opt/miniconda3/envs/spacy/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
df = pd.read_csv("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/anime-dataset-2023.csv")
df2 = pd.read_csv("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/web_scraping/imdb_scraped_datav2.csv", index_col=0)
df2 = df2.drop_duplicates(subset=['imdb_id']).reset_index(drop = True)

In [5]:
# Function to lowercase words and remove punctuation
def clean_text(text):
    return ' '.join(re.findall(r'\b\w+\b', text.lower()))

# Function to remove diacritics
def remove_diacritics(text):
    return unidecode(text)

    clean_text = html.unescape('Ki ga tsukeba &apos;Oyakusoku&apos;')


In [6]:
df['name_cleaned'] = df['Name'].apply(clean_text)
df['name_cleaned'] = df['name_cleaned'].apply(remove_diacritics)
df2['name_cleaned'] = df2['title'].apply(clean_text)
df2['name_cleaned'] = df2['name_cleaned'].apply(remove_diacritics)

In [10]:
matches = []
threshold = 70  # Adjust the threshold as needed

# Split the dataframe into chunks
chunk_size = len(df) // 10 + 1
chunks = [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

In [35]:
df2

,title,description,url,image,rating_value,genre,content_rating,creator,main_cast,keywords,duration,soundtrack_trackname,lyrics,music,arrangement,performed_by,imdb_id,name_cleaned
0,Kingdom,"In ancient China, a young boy dreams of becomi...",https://www.imdb.com/title/tt2404499/,https://m.media-amazon.com/images/M/MV5BZjQzN2...,8.6,"['Animation', 'Action', 'Drama']",TV-MA,"[{'@type': 'Organization', 'url': 'https://www...","[{'@type': 'Person', 'url': 'https://www.imdb....","china,chinese history,anime animation,epic act...",NaN,NaN,NaN,NaN,NaN,NaN,tt2404499,kingdom
1,A Determined Crossing,The Hi Shin Unit&apos;s mission is clear: cros...,https://www.imdb.com/title/tt31248682/,https://m.media-amazon.com/images/M/MV5BNTQ1ZG...,9.5,"['Animation', 'Action', 'Drama']",NaN,"[{'@type': 'Organization', 'url': 'https://www...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt31248682,a determined crossing
2,Eiga Doraemon: Nobita no Chikyû Symphony,Doraemon and friends go on an adventure to mee...,https://www.imdb.com/title/tt29402528/,https://m.media-amazon.com/images/M/MV5BNDQ2Nj...,6.8,"['Animation', 'Adventure', 'Comedy']",NaN,"[{'@type': 'Organization', 'url': 'https://www...","[{'@type': 'Person', 'url': 'https://www.imdb....","based on tv series,based on anime,based on ani...",PT1H55M,NaN,NaN,NaN,NaN,NaN,tt29402528,eiga doraemon nobita no chikyu symphony
3,Sôsô no Furîren,An elf and her friends defeat a demon king in ...,https://www.imdb.com/title/tt22248376/,https://m.media-amazon.com/images/M/MV5BODU1NT...,9.0,"['Animation', 'Adventure', 'Drama']",TV-14,"[{'@type': 'Organization', 'url': 'https://www...","[{'@type': 'Person', 'url': 'https://www.imdb....","shounen,based on manga,2d animation,anime anim...",NaN,Yuusha,(Hero),[Opening Theme],Music by Ayase,Arranged by Ayase,tt22248376,soso no furiren
4,The smell of the battlefield,"As soon as hostilities begin, nothing goes acc...",https://www.imdb.com/title/tt30989087/,https://m.media-amazon.com/images/M/MV5BYTVlMz...,9.3,"['Animation', 'Action', 'Drama']",NaN,"[{'@type': 'Organization', 'url': 'https://www...",NaN,anime,NaN,NaN,NaN,NaN,NaN,NaN,tt30989087,the smell of the battlefield
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18943,Kuma ni kuwarenu otoko,"Morality tale, musical, involving two boys who...",https://www.imdb.com/title/tt0422605/,https://m.media-amazon.com/images/M/MV5BN2MwYT...,5.4,"['Animation', 'Short']",NaN,"[{'@type': 'Organization', 'url': 'https://www...",NaN,NaN,PT9M,NaN,NaN,NaN,NaN,NaN,tt0422605,kuma ni kuwarenu otoko
18944,Suteneko Tora-chan,A family of cats find a homeless kitten and ta...,https://www.imdb.com/title/tt0419174/,https://m.media-amazon.com/images/M/MV5BNzA1Y2...,7.0,"['Animation', 'Short']",NaN,"[{'@type': 'Organization', 'url': 'https://www...",NaN,"anime animation,animal protagonist,milk,runawa...",PT21M,NaN,NaN,NaN,NaN,NaN,tt0419174,suteneko tora chan
18945,Kumo no ito,"Based on a short story by Akutagawa Ryunosuke,...",https://www.imdb.com/title/tt0418812/,https://m.media-amazon.com/images/M/MV5BYmNmNz...,7.0,"['Animation', 'Short', 'Fantasy']",NaN,"[{'@type': 'Organization', 'url': 'https://www...",NaN,"spiderweb,anime animation,voice over narration...",PT10M,NaN,NaN,NaN,NaN,NaN,tt0418812,kumo no ito
18946,Musume Dôjôji,Another of the multitudinous versions of Dojoj...,https://www.imdb.com/title/tt0038759/,https://m.media-amazon.com/images/M/MV5BMmMzMm...,6.6,"['Animation', 'Short']",NaN,"[{'@type': 'Organization', 'url': 'https://www...",NaN,NaN,PT20M,NaN,NaN,NaN,NaN,NaN,tt0038759,musume dojoji


In [19]:
chunks[0]

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL,name_cleaned
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...,cowboy bebop
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...,cowboy bebop tengoku no tobira
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...,trigun
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...,witch hunter robin
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...,bouken ou beet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,2712,Shiratori Reiko de Gozaimasu!,I am THE Shiratori Reiko!,白鳥麗子でございます！,5.31,"Comedy, Romance","19-year old Reiko is a rich, arrogant, materia...",OVA,1.0,"Apr 25, 1990",...,Manga,44 min,PG-13 - Teens 13 or older,11335.0,13806,0,184.0,699,https://cdn.myanimelist.net/images/anime/3/832...,shiratori reiko de gozaimasu
2487,2713,Rockman.EXE Beast+,UNKNOWN,ロックマンエグゼビースト+（プラス）,6.79,"Action, Adventure, Sci-Fi","Beast+ continues from where Beast left off, th...",TV,26.0,"Apr 8, 2006 to Sep 30, 2006",...,Game,10 min per ep,PG - Children,4876.0,7320,5,2106.0,6229,https://cdn.myanimelist.net/images/anime/12/73...,rockman exe beast
2488,2714,Sotsugyou: Graduation,Graduation,卒業~Graduation~,6.08,Drama,Regardless of how students handle the many aca...,OVA,2.0,"Jan 21, 1995 to Feb 21, 1995",...,Visual novel,30 min per ep,PG-13 - Teens 13 or older,8489.0,11187,3,516.0,1658,https://cdn.myanimelist.net/images/anime/1855/...,sotsugyou graduation
2489,2715,Totsuzen! Neko no Kuni Banipal Witt,Catnapped,とつぜん！猫の国 バニパルウィット,6.74,Adventure,"Fifth-grader, Toriyasu, and his little sister,...",Movie,1.0,"May 30, 1998",...,Novel,1 hr 16 min,G - All Ages,5109.0,9667,11,1005.0,2801,https://cdn.myanimelist.net/images/anime/6/582...,totsuzen neko no kuni banipal witt


In [17]:
def compute_matches_chunk(df_chunk, df2, threshold=80):
    matches_chunk = []
    for index1, row1 in tqdm(df_chunk.iterrows(), total=len(df_chunk), desc="Processing df"):
        for index2, row2 in tqdm(df2.iterrows(), total=len(df2), desc="Processing df2", leave=False):
            similarity = fuzz.token_set_ratio(row1["name_cleaned"], row2["name_cleaned"])
            if similarity >= threshold:
                matches_chunk.append((row1["name_cleaned"], row2["name_cleaned"], similarity))
    return matches_chunk

In [21]:
matches = compute_matches_chunk(chunks[0], df2, threshold=70)


Processing df: 100%|██████████| 2491/2491 [1:06:26<00:00,  1.60s/it]


In [36]:
matches2 = compute_matches_chunk(chunks[1], df2, threshold=70)

Processing df: 100%|██████████| 2491/2491 [1:07:56<00:00,  1.64s/it]


In [37]:
matches3 = compute_matches_chunk(chunks[2], df2, threshold=70)

Processing df: 100%|██████████| 2491/2491 [1:09:01<00:00,  1.66s/it]
